# Classification CIFAR-10

In [1]:
import tensorflow as tf
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Load Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Print dimensions of training data
print("Training data shape: ", x_train.shape)
print("Training labels shape: ", y_train.shape)

# Print dimensions of test data
print("Test data shape: ", x_test.shape)
print("Test labels shape: ", y_test.shape)

# Print number of unique classes
num_classes = len(np.unique(y_train))
print("Number of classes: ", num_classes)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000, 1)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000, 1)
Number of classes:  10


#### Data Augmentation

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images by 10 degrees
    width_shift_range=0.1,  # randomly shift images horizontally by 10% of the width
    height_shift_range=0.1,  # randomly shift images vertically by 10% of the height
    zoom_range=0.1,  # randomly zoom images by 10%
    horizontal_flip=True  # randomly flip images horizontally
)

data_gen_batch_size = 64

In [4]:
# # Fit the ImageDataGenerator on x_train
# datagen.fit(x_train)

# # Generate augmented images
# augmented_images = datagen.flow(x_train, y_train, batch_size=data_gen_batch_size)

# # Iterate over the augmented images and append them to x_train
# for i, (x_batch, y_batch) in enumerate(augmented_images):
#     x_train = np.concatenate((x_train, x_batch), axis=0)
#     y_train = np.concatenate((y_train, y_batch), axis=0)
#     if i >= len(x_train) // data_gen_batch_size:
#         break

# # Print the new dimensions of x_train
# print("New dimensions of x_train: ", x_train.shape)
# print("New dimensions of y_train: ", y_train.shape)


### Preprocess data

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
# Settings
resize = (224, 224)

# One hot encode labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load as tf dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Normalize images
def normalize_img(image, label):

    # Resize images
    image = tf.image.resize(image, resize)

    return tf.cast(image, tf.float32) / 255., label

# Normalise images
ds_train = train_dataset.map(normalize_img)
ds_test = test_dataset.map(normalize_img)

## Declare model

In [7]:
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Dropout, \
    AveragePooling2D, BatchNormalization, Activation, Add, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomZoom, RandomFlip, RandomTranslation

In [8]:
# Define initializer
initializer = 'he_normal' # tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=42)
classification_initializer = 'glorot_normal'

activation_function = 'relu' # 'selu'
use_bias = False # Batch Norm takes care of it
epsilon_bn = 1.001e-5
kernel_reg = None # tf.keras.regularizers.l1_l2(0.01)

### ResNet Blocks

#### Identity Block

In [9]:
# Define resnet identity block

def identity_block(filter, kernel_size=3):
    def _identity_block(x):
        input_x = x

        # Layer 1
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)
        x = Activation(activation_function)(x)

        # Layer 2
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)

        # Add Residue
        x = Add()([x, input_x])
        
        # Activation
        x = Activation(activation_function)(x)

        return x
    
    return _identity_block

#### Convolutional Res Block

In [10]:
# Define resnet convolutional block

def convolutional_res_block(filter, kernel_size=3):
    def _convolutional_res_block(x):
        input_x = x

        # Layer 1
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)
        x = Activation(activation_function)(x)

        # Layer 2
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)

        # Layer 3
        input_x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(input_x)
        input_x = BatchNormalization(axis=3, epsilon=epsilon_bn)(input_x)

        # Add Residue
        x = Add()([x, input_x])
        
        # Activation
        x = Activation(activation_function)(x)

        return x
    
    return _convolutional_res_block

#### Reduction Convolutional Res Block

In [11]:
# Define reduction convolutional block

def reduction_convolutional_res_block(filter, kernel_size=3, strides=2):
    def _reduction_convolutional_res_block(x):
        input_x = x

        # Layer 1
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)
        x = Activation(activation_function)(x)

        # Layer 2
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, strides=strides, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)

        # Layer 3
        input_x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, strides=strides, kernel_regularizer=kernel_reg)(input_x)
        input_x = BatchNormalization(axis=3, epsilon=epsilon_bn)(input_x)

        # Add Residue
        x = Add()([x, input_x])

        x = Activation(activation_function)(x)

        return x
    
    return _reduction_convolutional_res_block

### Other Blocks

#### Convolutional Block

In [12]:
# Convolutional block
def convolutional_block(filter, kernel_size=3, strides=2):
    def _convolutional_block(x):
        
        x = Conv2D(filter, kernel_size=kernel_size, 
                   padding = 'same', kernel_initializer=initializer,
                   use_bias=use_bias, strides=strides, kernel_regularizer=kernel_reg)(x)
        x = BatchNormalization(axis=3, epsilon=epsilon_bn)(x)
        x = Activation(activation_function)(x)

        return x
    
    return _convolutional_block

### Model definition

In [13]:
model_name = 'model'

#### Data augmentation

In [14]:
data_aug = [
    RandomRotation(0.3, seed=42),
    RandomTranslation(0.4, 0.4, seed=42),
    RandomZoom(0.3, seed=42),
    RandomFlip(seed=42),
]

In [15]:
data_aug = [
    RandomRotation(0.1, seed=42),
    RandomTranslation(0.1, 0.1, seed=42),
    RandomZoom(0.1, seed=42),
    RandomFlip(seed=42),
]

#### Residual Nets

In [16]:
model_name = 'resnet_1_data_augmentation'

conv_net = [
    convolutional_block(64),
    identity_block(64),
    reduction_convolutional_res_block(128),
    identity_block(128),
    reduction_convolutional_res_block(256),
    identity_block(256),
    reduction_convolutional_res_block(512),
    identity_block(512),
    reduction_convolutional_res_block(512),
    reduction_convolutional_res_block(512),
    AveragePooling2D(pool_size=4, strides=4, padding='same'),
]

In [17]:
model_name = 'resnet_3'

conv_net = [
    convolutional_block(64),
    identity_block(64),
    identity_block(64),
    AveragePooling2D(pool_size=2, strides=2, padding='same'),
    convolutional_res_block(128),
    identity_block(128),
    identity_block(128),
    AveragePooling2D(pool_size=2, strides=2, padding='same'),
    convolutional_res_block(256),
    identity_block(256),
    identity_block(256),
    AveragePooling2D(pool_size=2, strides=2, padding='same'),
    convolutional_res_block(512, kernel_size=1),
    identity_block(512, kernel_size=1),
    identity_block(512, kernel_size=1),
    AveragePooling2D(pool_size=2, strides=2, padding='same')
]

In [18]:
model_name = 'resnet_4'

conv_net = [
    convolutional_block(64, kernel_size = 7, strides = 2),
    AveragePooling2D(pool_size=3, strides = 2, padding = 'same'),
    
    identity_block(64),

    AveragePooling2D(pool_size=(2, 2), padding = 'same'),

    convolutional_res_block(128),

    AveragePooling2D(pool_size=(2, 2), padding = 'same'),

    convolutional_res_block(256),

    AveragePooling2D(pool_size=(4, 4), padding = 'same'),

    convolutional_res_block(512),
    
    convolutional_block(512, kernel_size = 3, strides = 2),
    AveragePooling2D(pool_size=2, strides = 2, padding = 'same'),
]

In [19]:
model_name = 'resnet_small_tests'

conv_net = [
    convolutional_block(64, kernel_size = 7, strides = 2),

    identity_block(64),

    AveragePooling2D(pool_size=2, padding = 'same'),

    convolutional_res_block(128),

    AveragePooling2D(pool_size=2, padding = 'same'),

    convolutional_res_block(256),

    AveragePooling2D(pool_size=4, padding = 'same'),
]

In [20]:
model_name = 'resnet_4_32'

conv_net = [
    convolutional_block(64, kernel_size = 7, strides = 2),

    identity_block(64),

    AveragePooling2D(pool_size=(2, 2), padding = 'same'),

    convolutional_res_block(128),

    convolutional_res_block(256),

    AveragePooling2D(pool_size=(4, 4), padding = 'same'),

    convolutional_res_block(512),
    
    convolutional_block(512, kernel_size = 1, strides = 1),
    AveragePooling2D(pool_size=2, strides = 2, padding = 'same'),
]

#### ResNet 18

In [21]:
model_name = 'resnet_5'

conv_net = [
    convolutional_block(64, kernel_size = 7, strides = 2),
    MaxPooling2D(pool_size=3, strides = 2, padding = 'same'),

    identity_block(64),
    identity_block(64),

    convolutional_block(128),
    identity_block(128),

    convolutional_block(256),
    identity_block(256),

    convolutional_block(512),
    identity_block(512),

    AveragePooling2D(pool_size=7)
]

In [22]:
load_model_name = None
if False:
    model_name = 'resnet_4_1'
    load_model_name = 'resnet_4'

In [23]:
classification_net = [
    Flatten(),
    Dense(512, kernel_initializer=initializer, activation=activation_function),
    Dropout(0.5),
    Dense(num_classes, kernel_initializer=classification_initializer, activation='softmax')
]

### Compile mondel

In [24]:
# Define input layer
input_shape = (resize[0], resize[1], 3)
input_layer = Input(shape=input_shape)

In [25]:
def create_classifier():

    def compile_layers(input, layers):
        for layer in layers:
            input = layer(input)
        return input
    
    # Build Data augmentation
    data_augmentation_layers = compile_layers(input_layer, data_aug)

    # Build Feature Extractor
    conv_net_layers = compile_layers(data_augmentation_layers, conv_net)

    # Build Classifier
    classification_layers = compile_layers(conv_net_layers, classification_net)

    # Build model
    model = Model(inputs=input_layer, outputs=classification_layers)

    return model

In [26]:
# Create model
model = create_classifier()

In [27]:
if load_model_name is not None:
    model.load_weights('models/' + load_model_name + '.h5')  

In [28]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
random_rotation_1 (RandomRotati (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
random_translation_1 (RandomTra (None, 224, 224, 3)  0           random_rotation_1[0][0]          
__________________________________________________________________________________________________
random_zoom_1 (RandomZoom)      (None, 224, 224, 3)  0           random_translation_1[0][0]       
_______________________________________________________________________________________

## Train/Test model

In [29]:
# Train settings
epochs = 200
batch_size = 64 # 8

# Define optimizer
learning_rate = 0.005
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

### Prepare data

In [30]:
# Configure Dataset for Performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

# ds_train = ds_train.cache()
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)

ds_test = ds_test.batch(batch_size)
ds_test = ds_test.prefetch(buffer_size=AUTOTUNE)

del x_train, y_train, x_test, y_test

### Run train/test

In [31]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

import datetime

In [32]:
def train_test_model(model):

    # Compile model
    model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(label_smoothing=0.2), metrics=['accuracy'])

    # Learning Rate Scheduler
    def scheduler(epoch, lr):
        if epoch < 50:
            return lr
        else:
            return lr * tf.math.exp(-0.1)

    # Save model callback
    checkpoint = ModelCheckpoint('models/' + model_name + '.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    # Tensorboard callback
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '_' + model_name
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    print('tensorboard --logdir ' + log_dir)

    # Train model
    run_hist = model.fit(ds_train, validation_data=ds_test,
                         epochs=epochs, batch_size=batch_size, 
                         
                         callbacks=[checkpoint, tensorboard_callback])
    
    return run_hist

In [33]:
# Run train and validation
run_hist = train_test_model(model)

tensorboard --logdir logs/fit/20231123-190252_resnet_5
Epoch 1/200


  1/782 [..............................] - ETA: 0s - loss: 3.2872 - accuracy: 0.0469WARNING:tensorflow:From c:\Users\benoi\AppData\Local\pypoetry\Cache\virtualenvs\tp-8inf919-aDMhdl4N-py3.8\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
782/782 [==============================] - ETA: 0s - loss: 2.1789 - accuracy: 0.2162
Epoch 00001: val_accuracy improved from -inf to 0.18630, saving model to models\resnet_5.h5
782/782 [==============================] - 195s 249ms/step - loss: 2.1789 - accuracy: 0.2162 - val_loss: 2.2511 - val_accuracy: 0.1863
Epoch 2/200
782/782 [==============================] - ETA: 0s - loss: 2.0412 - accuracy: 0.2877
Epoch 00002: val_accuracy improved from 0.18630 to 0.31000, saving model to models\resnet_5.h5
782/782 [==============================] - 190s 243ms/step - loss: 2.0412

In [ ]:
# Run test
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print(f'Test accuracy: {test_acc}')

## Plot history

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy and loss
plt.plot(run_hist.history['accuracy'], label='train')
plt.plot(run_hist.history['val_accuracy'], label='test')
plt.plot(run_hist.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()